In [8]:
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re

In [9]:
# Download English module

!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 4.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/12.8 MB 2.8 MB/s eta 0:00:05
     ----- ---------------------------------- 1.8/12.8 MB 2.7 MB/s eta 0:00:05
     ------- -------------------------------- 2.4/12.8 MB 2.6 MB/s eta 0:00:04
     ------- -------------------------------- 2.4/12.8 MB 2.6 MB/s eta 0:00:04
     ---------- ----------------------------- 3.4/12.8 MB 2.5 MB/s eta 0:00:04
     ------------ --------------------------- 3.9/12.8 MB 2.6 MB/s eta 0:00:04
     ------------- -------------------------- 4.2/12.8 MB 2.5 MB/s eta 0:00:04
     ------------- -------------------------- 4.5/12.8 MB 2.4 MB/s eta 0:00:04
     -------------- ------------------------- 4.7/12.8 MB 2.3 MB/s eta 0:00:04
     --------------- ------------------------ 5.0/12.8 MB 2.1 MB/s

In [10]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

## Load 20th Century file

In [12]:
# Load the book

with open('20th_century.txt', 'r', errors='ignore') as file:
    data = file.read().replace('\n', '')

In [13]:
book = NER(data)

In [14]:
# Visualize identified entities

displacy.render(book[273:500], style = "ent", jupyter = True)

## Get named entity list per sentence

In [17]:
df_sentences = [] # empty shell to store results

# Loop through sentences, get entity list for each sentence
for sent in book.sents:
    entity_list = [ent.text for ent in sent.ents]
    df_sentences.append({"sentence": sent, "entities": entity_list})
    
df_sentences = pd.DataFrame(df_sentences)

In [18]:
df_sentences.head(10)

,sentence,entities
0,"(Key, events, of, the, 20th, century, -, Wikip...","[the 20th century - WikipediaJump, Contribute,..."
1,"(accountLog, inPersonal, toolsDonate, Create, ...","[20th, the 20th century]"
2,"(World, War, I, (, 1914â€“1918)1.1.2Russian, R...",[World War I]
3,"(depression1.2.2The, rise, of, dictatorship1.3...","[World War II, Pacific1.3.7.1Background1.3.8Ja..."
4,"(begins1.4The, post, -, war, world1.4.1The, en...","[Cold War, 1947â€“1991)1.4.3War, the Cold War1..."
5,"(What, links, hereRelated, changesUpload, file...","[pageGet, URLDownload, Download, Wikipedia, en..."
6,"(The, World, Wars, sparked, tension, between, ...","[the Cold War, the Space Race]"
7,"(These, advancements, have, played, a, signifi...","[the 21st century, today]"
8,"(Historic, events, in, the, 20th, century[edit...",[the 20th]
9,"(Edwardian, eraThe, new, beginning, of, the, 2...","[Edwardian, the 20th century]"


## Load Country names

In [20]:
# Import characters

countries_df = pd.read_csv("countries_list_20th_century_1.5.csv", index_col = 0)

In [21]:
countries_df.head()

,country_name
1,Afghanistan
2,Albania
3,Algeria
4,Andorra
5,Angola


## Filtering entities from the text

In [23]:
# Function to filter out entities not of interest

def filter_entity(ent_list, countries_df):
    return [ent for ent in ent_list 
            if ent in list(countries_df['country_name'])]

In [58]:
def filter_entity(ent_list, countries):
    return[ent for ent in ent_list
          if ent in list(countries['country_name'])]

In [60]:
df_sentences['country_entities']=df_sentences['entities'].apply(lambda x: filter_entity(x,countries_df))

In [62]:
df_sentences.head(10)

,sentence,entities,country_entities
0,"(Key, events, of, the, 20th, century, -, Wikip...","[the 20th century - WikipediaJump, Contribute,...",[]
1,"(accountLog, inPersonal, toolsDonate, Create, ...","[20th, the 20th century]",[]
2,"(World, War, I, (, 1914â€“1918)1.1.2Russian, R...",[World War I],[]
3,"(depression1.2.2The, rise, of, dictatorship1.3...","[World War II, Pacific1.3.7.1Background1.3.8Ja...",[]
4,"(begins1.4The, post, -, war, world1.4.1The, en...","[Cold War, 1947â€“1991)1.4.3War, the Cold War1...",[]
5,"(What, links, hereRelated, changesUpload, file...","[pageGet, URLDownload, Download, Wikipedia, en...",[]
6,"(The, World, Wars, sparked, tension, between, ...","[the Cold War, the Space Race]",[]
7,"(These, advancements, have, played, a, signifi...","[the 21st century, today]",[]
8,"(Historic, events, in, the, 20th, century[edit...",[the 20th],[]
9,"(Edwardian, eraThe, new, beginning, of, the, 2...","[Edwardian, the 20th century]",[]


In [64]:
# Check

filter_entity(['Afghanistan'], countries_df)

[]

In [66]:
df_sentences_filter=df_sentences[df_sentences['country_entities'].map(len) > 0]

In [68]:
df_sentences_filter.head()

,sentence,entities,country_entities


In [70]:
print(countries_df.columns)


Index(['country_name'], dtype='object')
